In [11]:
import numpy as np
import h5py

In [12]:
hf = h5py.File('/content/drive/MyDrive/data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5','r')
x = hf.get('X')[:50000]
y = hf.get('y')[:50000]

x_electron = np.asarray(x)
y_electron = np.asarray(y)

In [13]:
hf1 = h5py.File('/content/drive/MyDrive/data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5','r')
x1 = hf1.get('X')[:50000]
y1 = hf1.get('y')[:50000]

x_photon = np.asarray(x1)
y_photon = np.asarray(y1)

In [14]:
x_data = np.concatenate((x_electron,x_photon),axis=0)
y_data = np.concatenate((y_electron,y_photon),axis=0)
avg_channel = np.mean(x_data[:,:, :, :2], axis=-1, keepdims=True)

# Concatenate the average channel with the original image
x_data = np.concatenate((x_data, avg_channel), axis=-1)

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
x_train, x_validate, y_train, y_validate = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_validate, y_validate, test_size=0.5, random_state=42)

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the number of classes in the classification task
num_classes = 1

# Define the patch size and number of patches
patch_size = 4
num_patches = (32 // patch_size) ** 2

# Define the embedding dimension and transformer block parameters
embedding_dim = 64
transformer_units = [
    embedding_dim * 2,
    embedding_dim,
]

# Define the input layer for the image
inputs = layers.Input(shape=(32, 32, 3))

# Split the image into patches
patches = layers.Reshape((num_patches, patch_size * patch_size * 3))(inputs)

# Add a learnable embedding to each patch vector
embedding_layer = layers.Dense(embedding_dim)
embedded_patches = embedding_layer(patches)

# Add positional encoding to the embedded patches
positional_encoding_layer = layers.Embedding(input_dim=num_patches, output_dim=embedding_dim)
positions = tf.range(start=0, limit=num_patches, delta=1)
position_encodings = positional_encoding_layer(positions)
encoded_patches = embedded_patches + position_encodings

# Apply a stack of transformer blocks
for units in transformer_units:
    transformer_layer = layers.MultiHeadAttention(num_heads=8, key_dim=embedding_dim // 8)
    x1 = transformer_layer(encoded_patches, encoded_patches)
    x1 = layers.Dropout(0.1)(x1)
    x2 = layers.Add()([x1, encoded_patches])
    x3 = layers.LayerNormalization()(x2)
    x4 = layers.Dense(units, activation="relu")(x3)
    x5 = layers.Dropout(0.1)(x4)
    x6 = layers.Dense(embedding_dim)(x5)
    x7 = layers.Dropout(0.1)(x6)
    encoded_patches = layers.Add()([x7, x2])
    encoded_patches = layers.LayerNormalization()(encoded_patches)

# Apply global average pooling to obtain a single feature vector
features = layers.GlobalAveragePooling1D()(encoded_patches)

# Add a classification output layer
outputs = layers.Dense(num_classes, activation="sigmoid")(features)

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)

# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 reshape_1 (Reshape)            (None, 64, 48)       0           ['input_2[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 64, 64)       3136        ['reshape_1[0][0]']              
                                                                                                  
 tf.__operators__.add_1 (TFOpLa  (None, 64, 64)      0           ['dense_6[0][0]']                
 mbda)                                                                                      

In [18]:
from keras.optimizers import Adam

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20)

Epoch 1/20
2500/2500 [==============================] - 32s 10ms/step - loss: 0.6667 - accuracy: 0.5893 - val_loss: 0.6465 - val_accuracy: 0.6326
Epoch 2/20
2500/2500 [==============================] - 24s 10ms/step - loss: 0.6378 - accuracy: 0.6429 - val_loss: 0.6370 - val_accuracy: 0.6337
Epoch 3/20
2500/2500 [==============================] - 24s 9ms/step - loss: 0.6237 - accuracy: 0.6584 - val_loss: 0.6350 - val_accuracy: 0.6523
Epoch 4/20
2500/2500 [==============================] - 24s 10ms/step - loss: 0.6137 - accuracy: 0.6705 - val_loss: 0.6126 - val_accuracy: 0.6703
Epoch 5/20
2500/2500 [==============================] - 26s 10ms/step - loss: 0.6058 - accuracy: 0.6795 - val_loss: 0.6070 - val_accuracy: 0.6803
Epoch 6/20
2500/2500 [==============================] - 26s 10ms/step - loss: 0.6021 - accuracy: 0.6819 - val_loss: 0.6126 - val_accuracy: 0.6675
Epoch 7/20
2500/2500 [==============================] - 25s 10ms/step - loss: 0.5987 - accuracy: 0.6878 - val_loss: 0.6146 - 

In [19]:
from sklearn.metrics import roc_auc_score
pred_prob1 = model.predict(x_test)
auc_score1 = roc_auc_score(y_test, pred_prob1[:])
auc_score1

313/313 [==============================] - 2s 5ms/step


0.7651813666322997